In [1]:
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.models import Sequential, model_from_json
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import pickle
import matplotlib.pyplot as plt
import numpy as np
#import cv2

In [2]:
train_datagen = ImageDataGenerator(
    rescale = 1/255,
    rotation_range = 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)

val_datagen = ImageDataGenerator(rescale = 1/255)


In [3]:
train_it = train_datagen.flow_from_directory(
    'cardataset/train/', 
    class_mode='categorical', 
    batch_size=32, 
    target_size = (150,150))

val_it = val_datagen.flow_from_directory(
    'cardataset/test/', 
    class_mode='categorical', 
    batch_size=32,
    target_size = (150,150))

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'cardataset/train/'

In [4]:
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation = 'relu', input_shape= (150,150,3)),
    layers.MaxPooling2D((2,2)),
    
    layers.Conv2D(64, (3,3), activation = 'relu'),
    layers.MaxPooling2D((2,2)),
    layers.Dropout(0.2),
    
    layers.Conv2D(128, (3,3), activation = 'relu'),
    layers.MaxPooling2D((2,2)),
    layers.Dropout(0.2),
    
    layers.Conv2D(128, (3,3), activation = 'relu'),
    layers.MaxPooling2D((2,2)),
    layers.Dropout(0.4),

    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),

    layers.Dense(8, activation='softmax')
])

          
model.compile(loss = "categorical_crossentropy",
             optimizer = "adam",
             metrics = ['accuracy'])

In [5]:
model.fit_generator(train_it,
          validation_data=val_it, 
          epochs= 100,
          #steps_per_epoch = 665, #datasize/batchsize = 21284/32
          #callbacks=[callback]
                   )

Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 666 steps, validate for 137 steps
Epoch 1/100
666/666 [==============================] - 1248s 2s/step - loss: 1.5846 - accuracy: 0.4101 - val_loss: 1.5127 - val_accuracy: 0.4744
Epoch 2/100
666/666 [==============================] - 1329s 2s/step - loss: 1.3978 - accuracy: 0.4873 - val_loss: 1.2995 - val_accuracy: 0.5354
Epoch 3/100
666/666 [==============================] - 1271s 2s/step - loss: 1.2921 - accuracy: 0.5292 - val_loss: 1.3562 - val_accuracy: 0.5210
Epoch 4/100
666/666 [==============================] - 1254s 2s/step - loss: 1.2171 - accuracy: 0.5596 - val_loss: 1.1889 - val_accuracy: 0.5781
Epoch 5/100
666/666 [==============================] - 1202s 2s/step - loss: 1.1649 - accuracy: 0.5791 - val_loss: 1.2239 - val_accuracy: 0.5525
Epoch 6/100
666/666 [==============================] - 820s 1s/step - loss: 1.1141 - accuracy: 0.6019 - v

In [6]:
saved_model = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(saved_model)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [7]:
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [11]:
loaded_model.compile(loss = "categorical_crossentropy",
             optimizer = "adam",
             metrics = ['accuracy'])
score = loaded_model.evaluate(train_it, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

  ...
    to  
  ['...']
accuracy: 82.53%
